# Create features

In [1]:
# def create_features(years, lags_ID, lags_DA, lags_VOL):

#     import pandas as pd

#     ########################################################################################################
#     # Intraday price (ID)

#     ID = pd.read_pickle(f"./ID.pkl")

#     ID = ID.astype('float')

#     ID = ID.loc[ID.index.year.isin(years)]

#     ########################################################################################################
#     # Day-ahead price (DA)

#     DA = pd.read_pickle(f"../epex/DA.pkl")

#     DA = DA.astype('float')

#     DA = DA.loc[DA.index.year.isin(years)]

#     features = pd.merge(ID, DA, left_index = True, right_index = True, how = 'outer')

#     ########################################################################################################
#     # Volume

#     features = features.rename(columns = {'Volume':'VOL'})

#     ########################################################################################################
#     # Load (LOAD)
    
#     # LOAD = pd.read_csv('./LOAD_NL.csv', index_col = 0)

#     # LOAD.index.names = ['Date']

#     # LOAD = LOAD.rename(columns = {'Forecast':'Load forecast'})

#     # LOAD.index = pd.to_datetime(LOAD.index)

#     # LOAD = LOAD.resample('1H').mean()

#     # features = pd.merge(features, LOAD, left_index = True, right_index = True, how = 'outer')

#     # features['LFE'] = (features['Load'] - features['Load forecast']).abs()

#     ########################################################################################################
#     # Load (LOAD)

#     LOAD = pd.read_csv('./LOAD.csv', index_col = 0)

#     features = pd.merge(features, LOAD, left_index = True, right_index = True, how = 'outer')

#     ########################################################################################################
#     # Load forecast (LOAD_F)
    
#     LOAD_F = pd.read_csv('./LOAD_F.csv', index_col = 0)

#     features = pd.merge(features, LOAD_F, left_index = True, right_index = True, how = 'outer')

#     ########################################################################################################
#     # Lags

#     for l in lags_ID:
#         features[f'ID3 ({l})'] = features['ID3'].shift(l)
    
#     for l in lags_DA:
#         features[f'MCP ({l})'] = features['MCP'].shift(l)

#     ########################################################################################################
#     # Lags volume

#     for l in lags_VOL:
#         features[f'VOL ({l})'] = features['VOL'].shift(l)

#     ########################################################################################################
#     # Generation

#     features.index = features.index.tz_localize('UTC')

#     GEN = pd.read_csv('./GEN.csv', index_col = 0)

#     features = pd.merge(features, GEN, left_index = True, right_index = True, how = 'outer')

#     ########################################################################################################
#     # Environment

#     ENV = pd.read_csv('./ENV.csv', index_col = 0)

#     features = pd.merge(features, ENV, left_index = True, right_index = True, how = 'outer')

#     ########################################################################################################
#     # Daily dummies

#     for d in range(0, 7):
#         features[f'DOW {d}'] = 0
#         features.loc[features.index.weekday == d, f'DOW {d}'] = 1

#     for h in range(0, 24):
#         features[f'HOD {h}'] = 0
#         features.loc[features.index.hour == h, f'HOD {h}'] = 1

#     ########################################################################################################

#     # features = features.dropna()

#     return ID, DA, features

In [1]:
import pandas as pd

years = [2015, 2016, 2017, 2018]

lags_ID = range(-4, -169, -1)
lags_DA = [i for i in range(5, -6, -1) if i not in [0]]
lags_VOL = [i for i in range(-4, -25, -1) if i not in [0]]

features = pd.DataFrame(index = pd.date_range(f'{years[0]}.01.01 00:00', f'{years[-1]}.12.31 23:00', freq = 'H'))

features.index = pd.to_datetime(features.index)

features.index = features.index.tz_localize('UTC')

# features.index = pd.date_range(f'{years[0]}.01.01 00:00', f'{years[-1]}.12.31 23:00', freq = 'H')

########################################################################################################
# Intraday price (ID)

ID = pd.read_pickle(f"./ID.pkl")

ID = ID.astype('float')

ID = ID.loc[ID.index.year.isin(years)]

ID.index = ID.index.tz_localize('UTC')

features = pd.merge(features, ID, left_index = True, right_index = True, how = 'left')

########################################################################################################
# Day-ahead price (DA)

DA = pd.read_pickle(f"../epex/DA.pkl")

DA = DA.astype('float')

DA = DA.loc[DA.index.year.isin(years)]

DA.index = DA.index.tz_localize('UTC')

features = pd.merge(features, DA, left_index = True, right_index = True, how = 'left')

########################################################################################################
# Volume

features = features.rename(columns = {'Volume':'VOL'})

########################################################################################################
# Load (LOAD)

# LOAD = pd.read_csv('./LOAD_NL.csv', index_col = 0)

# LOAD.index.names = ['Date']

# LOAD = LOAD.rename(columns = {'Forecast':'Load forecast'})

# LOAD.index = pd.to_datetime(LOAD.index)

# LOAD = LOAD.resample('1H').mean()

# features = pd.merge(features, LOAD, left_index = True, right_index = True, how = 'outer')

# features['LFE'] = (features['Load'] - features['Load forecast']).abs()

########################################################################################################
# Load (LOAD)

LOAD = pd.read_csv('./LOAD.csv', index_col = 0)

features = pd.merge(features, LOAD, left_index = True, right_index = True, how = 'left')

########################################################################################################
# Load forecast (LOAD_F)

LOAD_F = pd.read_csv('./LOAD_F.csv', index_col = 0)

features = pd.merge(features, LOAD_F, left_index = True, right_index = True, how = 'left')

########################################################################################################
# Load forecast error (LOAD_FE)

features['LOAD_FE'] = (features['LOAD'] - features['LOAD_F']).abs()

########################################################################################################
# Lags

features = features.dropna()

for l in lags_ID:
    features[f'ID3 ({l})'] = features['ID3'].shift(-l)

for l in lags_DA:
    features[f'MCP ({l})'] = features['MCP'].shift(-l)

########################################################################################################
# Lags volume

for l in lags_VOL:
    features[f'VOL ({l})'] = features['VOL'].shift(-l)

########################################################################################################
# Generation

GEN = pd.read_csv('./GEN.csv', index_col = 0)

features = pd.merge(features, GEN, left_index = True, right_index = True, how = 'left')

########################################################################################################
# Environment

ENV = pd.read_csv('./ENV.csv', index_col = 0)

features = pd.merge(features, ENV, left_index = True, right_index = True, how = 'left')

########################################################################################################
# Daily dummies

for d in range(0, 7):
    features[f'DOW {d}'] = 0
    features.loc[features.index.weekday == d, f'DOW {d}'] = 1

for h in range(0, 24):
    features[f'HOD {h}'] = 0
    features.loc[features.index.hour == h, f'HOD {h}'] = 1

########################################################################################################

features = features.dropna()

features.to_csv('./features.csv')

features.to_pickle(f'./features.pkl', protocol = 4)

# ENTSOE data

In [3]:
# import pandas as pd

# from entsoe import EntsoeRawClient
# from entsoe import EntsoePandasClient

# client = EntsoePandasClient(api_key = '9ee80e1b-31ad-41b3-addc-1533c2b8227f')

# years = [2015, 2016, 2017, 2018]

# start = pd.Timestamp(f'{years[0]}.01.01')
# end = pd.Timestamp(f'{years[-1]}.12.31')
# country_code = 'NL'  # Netherlands

# ########################################################################################################
# # Generation

# GEN = client.query_generation_forecast(country_code, start = start, end = end)

# GEN = GEN.to_frame()

# GEN = GEN.rename(columns = {0:'Gen. forecast'})

# GEN.index = GEN.index.tz_convert('UTC')

# GEN = GEN.resample('1H').mean()

# GEN.to_csv('./GEN.csv')

# ########################################################################################################
# # Environment

# ENV = client.query_wind_and_solar_forecast(country_code, start = start, end = end, psr_type = None)

# ENV = ENV.rename(columns = {'Solar':'Gen. (s) forecast', 'Wind Offshore':'Gen. (w_off) forecast', 'Wind Onshore':'Gen. (w_on) forecast'})

# ENV.index = ENV.index.tz_convert('UTC')

# ENV = ENV.resample('1H').mean()

# ENV.to_csv('./ENV.csv')

# ########################################################################################################
# # Load

# LOAD = client.query_load(country_code, start = start, end = end)

# LOAD = LOAD.to_frame()

# LOAD = LOAD.rename(columns = {0:'LOAD'})

# LOAD.index = LOAD.index.tz_convert('UTC')

# LOAD = LOAD.resample('1H').mean()

# LOAD.to_csv('./LOAD.csv')

# ########################################################################################################
# # Load forecast

# LOAD_F = client.query_load_forecast(country_code, start = start, end = end)

# LOAD_F = LOAD_F.to_frame()

# LOAD_F = LOAD_F.rename(columns = {0:'LOAD_F'})

# LOAD_F.index = LOAD_F.index.tz_convert('UTC')

# LOAD_F = LOAD_F.resample('1H').mean()

# LOAD_F.to_csv('./LOAD_F.csv')